In [55]:
%%writefile main.nf
#!/usr/bin/env nextflow

/* pipeline input parameters, update this to your data dir */
dicom_data = "$baseDir/sdk_data/b7/a9dbb4ae17338be1f6f9aa1dadeee1/*.dcm"


process dicom_to_png {
    
    label 'dcm2png'
    
    input:
    path dicom_files
    
    output:
    stdout emit: dicom_to_png_log
    path('*.png'), emit: png_files
    
    script:
    """
    #!/usr/bin/env python3

    import pydicom
    import argparse
    import numpy as np
    from PIL import Image
    import os

    dicom_input = '$dicom_files'
    
    def main(dicom_input):
        png_out = dicom_input.split('.dcm')[0] + '.png'
        dicom_dataset = pydicom.dcmread(dicom_input)
        transformed_image = dicom_dataset.pixel_array.astype(float)
        scaled_image = np.uint8((np.maximum(transformed_image, 0) / transformed_image.max()) * 255.0)
        final_image = Image.fromarray(scaled_image)
        final_image.save(png_out)

    if __name__ == '__main__':
        main(dicom_input)
        """
}

process extract_metadata {
    
    label 'ext_metadata'
    
    input:
    path dicom_files
    
    output:
    stdout emit: extract_metadata_log
    path('*.csv'), emit: csv_files
    
    script:
    """
    #!/usr/bin/env python3

    import pandas as pd
    import os
    import argparse
    from dicom_csv import join_tree
    
    dicom_input = '$dicom_files'
    metadata_csv = 'dicom-metadata.csv'

    def main(dicom_input, metadata_csv):
        metadata_df = join_tree('.', verbose=2)
        dicom_metadata_df = metadata_df.loc[metadata_df['PixelRepresentation'].notnull()]
        dicom_metadata_df.drop_duplicates(inplace=True)
        return dicom_metadata_df.to_csv(metadata_csv)

    if __name__ == '__main__':
        main(dicom_input, metadata_csv)    
    """

}


// Define the entry workflow (initial workflow for Nextflow to run)
workflow {
    // def dicom_files = Channel.fromPath(dicom_data).buffer(size: 2)
    dicom_files = Channel.fromPath(dicom_data)
    dicom_to_png(dicom_files)
    extract_metadata(dicom_files)
}

Overwriting main.nf


In [56]:
%%writefile nextflow.config

process {
    withLabel: dcm2png {
        executor = 'awsbatch'
        queue = 'placeholder'
        container = 'public.ecr.aws/l5b8a5z6/nextflow-approved:batch_poc2'
    }
    withLabel: ext_metadata {
        executor = 'awsbatch'
        queue = 'placeholder'
        container = 'public.ecr.aws/l5b8a5z6/nextflow-approved:batch_poc2'
    } 
}

aws {
    region = 'us-east-1'
    batch {
        cliPath = '/home/ec2-user/miniconda/bin/aws'
        jobRole = 'placeholder'
    }
}
workDir = 'placeholder'

docker.enabled = true

Overwriting nextflow.config


In [57]:
!nextflow run main.nf -dsl2

N E X T F L O W  ~  version 21.10.6
Launching `main.nf` [insane_hilbert] - revision: ef23479e74
Uploading local `bin` scripts folder to s3://qa-brh-planx-pla-net-nf-366143200747/aartiv-40uchicago-2eedu/tmp/b5/7bf55bc4f50589bc8aaad79617eb74/bin
[-        ] process > dicom_to_png     -
[-        ] process > extract_metadata -

[-        ] process > dicom_to_png     [  0%] 0 of 2
[-        ] process > extract_metadata [  0%] 0 of 2

executor >  awsbatch (4)
[c9/5d8513] process > dicom_to_png (1)     [  0%] 0 of 3
[ce/facacd] process > extract_metadata (2) [  0%] 0 of 4

executor >  awsbatch (8)
[6a/c02871] process > dicom_to_png (4)     [  0%] 0 of 4
[1c/52257a] process > extract_metadata (4) [  0%] 0 of 4

executor >  awsbatch (9)
[6a/c02871] process > dicom_to_png (4)     [  0%] 0 of 6
[4c/7b4bfe] process > extract_metadata (5) [  0%] 0 of 6

executor >  awsbatch (12)
[36/a12c64] process > dicom_to_png (6)     [  0%] 0 of 7
[c8/789b0e] process > extract_metadata (6) [  0%] 0 of 6

execu